<a href="https://colab.research.google.com/github/Gajendra-theDataEngineer004627/Test/blob/ChessChampionshipProject/DE_Solutions_GRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update -y

In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [4]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [5]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [7]:
!pip install -q findspark
import findspark
findspark.init()

In [8]:
from pyspark.sql import SparkSession
spark = (SparkSession
 .builder
 .appName("<app_name>")
 .getOrCreate())


In [ ]:
# function to doanload data files from the google drive
import gdown
def downloadfiles_fromFolder() :
    url = "https://drive.google.com/drive/folders/1QgWPHV_l25Ui9L7et8mkZohAOG59UTkQ"   # can also be used input()
    if url.split("/")[-1]== "?usp=sharing":
       url = url.replace("?usp=sharing","")
    gdown.download_folder(url, output="/content")

downloadfiles_fromFolder()

In [10]:
# Creating the datasets for each data files and understanding the data

chess_wc_history_game_info = spark.read.load("/content/chess_wc_history_game_info.csv",format="csv", sep=",", inferSchema="true", header="true")

chess_wc_history_moves = spark.read.load("/content/chess_wc_history_moves.csv",format="csv", sep=",", inferSchema="true", header="true")

eco_codes = spark.read.load("/content/eco_codes.csv",format="csv", sep=",", inferSchema="true", header="true")


In [11]:
# checking the top 10 records are properly loaded or not
# print(chess_wc_history_moves.head())
# print(eco_codes.head())
# print(chess_wc_history_game_info.head())

In [16]:
# Creating temporary view for data manupulations
chess_wc_history_moves.createOrReplaceTempView("chess_wc_history_movesTemp")
eco_codes.createOrReplaceTempView("eco_codesTemp")
chess_wc_history_game_info.createOrReplaceTempView("chess_wc_history_game_infoTemp")

# ----Creating the table from the temp view so that the tables can be accessible globally(accessed across different Spark sessions) #

query_main_Event  = """Select * from chess_wc_history_game_infoTemp where
                        event not like '%KO%' and
                        event not like '%k.o.%' """


# -------------WCh knock out(string with k.o.,KO ) are not included in main event-----------#
spark.sql(query_main_Event).createOrReplaceTempView("chess_wc_history_game_infoTemp1")


chess_wc_history_movesTempTable = spark.table("chess_wc_history_movesTemp")
eco_codesTempTable = spark.table("eco_codesTemp")
chess_wc_history_game_infoTempTable = spark.table("chess_wc_history_game_infoTemp1")

# Checking the validity of data in table form  using mysql like capabilities in SparkSql
#result1 = spark.sql("SELECT * FROM chess_wc_history_movesTemp limit 5")
#result1.show()
#result2 = spark.sql("SELECT * FROM eco_codesTemp")
#result2.show()
# result2.printSchema()    # --> to check schema
  # result3 = spark.sql("SELECT * from chess_wc_history_game_infoTemp limit 5 ")
  # result3.show()
 #spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
  # eco_codes.columns    # this is used to check the data frames columns
  #chess_wc_history_game_info.columns


query1 = """ WITH CTE_WInner_List AS
                (select winner , event,round , tournament_name,date_played,
                count(*) over (partition by winner,tournament_name order by date_played) as WinnerCount
                from chess_wc_history_game_infoTemp1
                where winner not like '%draw%'and
                tournament_name like 'WorldChamp%' )
                SELECT winner, tournament_name, MAX(WinnerCount) AS Score
                FROM CTE_Winner_List
                GROUP BY winner ,tournament_name
                order BY tournament_name DESC
"""
spark.sql(query1).createOrReplaceTempView("temp_view1")

final_query1 = """ with CTE_Final AS (
                select winner, tournament_name, Score ,
                ROW_NUMBER() OVER (partition by tournament_name order by Score DESC) as Row_num
                from temp_view1 )
                select winner, tournament_name  --, Score
                from CTE_Final
                where Row_num = 1
                order by tournament_name DESC
"""

#  -----------Sollution of 1st Problem ----------------#
resultTest1 = spark.sql(final_query1)
resultTest1.show()
# TO DOWNLOAD FILES TO THE CONTENT FOLDER OF COLAB
#resultTest1.coalesce(1).write.mode("overwrite").csv("/content/resultTest_prb1", header=True)
# -----------------------2nd Problem mitigation ----------------#
spark.sql(final_query1).createOrReplaceTempView("Second_problem")

second_problem_query1  = """
        select winner as Player_name,
        count(*) as number_of_wins from Second_problem
        group by winner
        order by number_of_wins DESC
"""
#  -----------Sollution of 2nd Problem ----------------#
resultTest2 = spark.sql(second_problem_query1)
resultTest2.show()
# TO DOWNLOAD FILES TO THE CONTENT FOLDER OF COLAB
#resultTest2.coalesce(1).write.mode("overwrite").csv("/content/resultTest_prb2", header=True)
# -----------------------3rd Problem mitigation ----------------#

third_problem_query0 = """
                with cte_popular_move as (
                select A.eco, A.eco_name, B.game_id,
                count(*) over(partition by A.eco, B.game_id  order by A.eco) as no_of_occurances
                from eco_codesTemp A left join chess_wc_history_game_infoTemp1 B
                on A.eco = B.eco )
                select eco, eco_name ,
                count(*) over(partition by eco_name order by eco) as occurance
                from cte_popular_move
                group by eco,eco_name
                order by occurance DESC
"""

test_res6 = spark.sql(third_problem_query0).createOrReplaceTempView("third_query_test0")
#test_res6.show()

third_problem_query1 = """with cte_test_final as (
                        select eco, eco_name,
                        occurance as number_of_occurences,
                        ROW_NUMBER() over( order by occurance DESC) as MostRow,
                        ROW_NUMBER() over( order by occurance ASC) as leastRow
                        from third_query_test0 )
                        select
                        eco,
                        eco_name,
                        number_of_occurences   --,MostRow, leastRow
                        from cte_test_final
                        where  MostRow =1 or leastRow = 1
                        order by number_of_occurences DESC
                        """

result_test3 = spark.sql(third_problem_query1)   #.createOrReplaceTempView("third_query_test")
result_test3.show()

# -----------------------4th Problem mitigation ----------------#

fourth_problem_query = """
                with cte_popular_move_win as (
                select A.eco, A.eco_name, B.game_id,B.winner_elo
                from eco_codesTemp A left join chess_wc_history_game_infoTemp1 B
                on A.eco = B.eco
                where B.winner_elo is not null
                )
                select eco, eco_name ,winner_elo,
                count(*) over (partition by eco order by eco) as occurance
                from cte_popular_move_win
                group by eco,eco_name,winner_elo
                order by occurance DESC
                --limit 1
"""
spark.sql(fourth_problem_query).createOrReplaceTempView("fourth_problem_query0")
#r = spark.sql("select * from fourth_problem_query0  order by occurance DEsc  limit 10")
#r.show()

fourth_problem_query1 = """
                with CTE_final AS (
                select eco, eco_name ,occurance,
                ROW_NUMBER() over(order by occurance DESC) as row_num
                from fourth_problem_query0 )
                select eco, eco_name  --, occurance, row_num
                from CTE_final
                where  row_num=1
                order by occurance DESC

"""
result_test4 = spark.sql(fourth_problem_query1 )
result_test4.show()

# -----------------------5th Problem mitigation ----------------#

#chess_wc_history_movesTempTable.show()
#eco_codesTempTable.show()
#chess_wc_history_game_infoTempTable.show()

chess_wc_history_game_infoTempTable.createOrReplaceTempView("chess_wc_history_game_infoTempTable1")
chess_wc_history_movesTempTable.createOrReplaceTempView("chess_wc_history_movesTempTable1")

#test  = spark.sql("select game_id, event ,")

# move_no , move_Sequence(even number shoub be there),game_id,event, tournament_name
#Longest and shortest game ever played in a world championship in terms of move
# Final result will have only two rows
fifth_problem_query0 = """
                        with cte_test_move as (
                        select
                        A.game_id,B.date_played,
                        A.move_no, A.move_no_pair ,
                        A.move_Sequence,
                        B.event, B.tournament_name,
                        B.winner,
                        B.loser,
                        B.winner_elo,
                        B.loser_elo,
                        ROW_NUMBER() over(partition by A.game_id, B.date_played order by A.move_Sequence DESC) as longest_move_rank
                        --ROW_NUMBER() over(partition by A.game_id, B.date_played order by A.move_Sequence ASC) as shortest_move_rank
                        from chess_wc_history_movesTempTable1 A left join chess_wc_history_game_infoTempTable1 B
                        on A.game_id =  B.game_id )
                        select game_id, date_played,
                        --move_no,
                        --longest_move_rank as move_rank,
                        move_no_pair,
                        move_Sequence, event,tournament_name,winner,loser,
                        ROW_NUMBER() over(order by move_no_pair DESC) as high_rank,
                        --ROW_NUMBER() over(order by move_no_pair ASC) as low_rank,
                        winner_elo,loser_elo
                        from cte_test_move
                        where longest_move_rank = 1 and move_no_pair%2 = 0               --in (1,2,3,4)
                         and event is not null
                        --group by game_id, date_played, move_no_pair ,move_no
                        order by move_no_pair DESC

"""
spark.sql(fifth_problem_query0).createOrReplaceTempView("fifth_problem_query1")

fifth_problem_query2 = """
                      WITH CTE_Final AS (
                      SELECT
                      game_id,
                      event,
                      tournament_name,
                      move_no_pair AS number_of_moves,
                      ROW_NUMBER() OVER ( ORDER BY move_no_pair DESC) AS longest_game_played,
                      ROW_NUMBER() OVER ( ORDER BY move_no_pair ASC) AS shortest_game_played,
                      move_sequence
                      FROM fifth_problem_query1
                      )
                    SELECT
                    game_id,
                    event,
                    tournament_name,
                    number_of_moves
                    FROM CTE_Final
                    WHERE longest_game_played = 1 OR shortest_game_played = 1
                    ORDER BY number_of_moves DESC """

result_test5 = spark.sql(fifth_problem_query2)
result_test5.show()

# -----------------------6th Problem mitigation ----------------#

#chess_wc_history_movesTempTable.show()
#eco_codesTempTable.show()
#chess_wc_history_game_infoTempTable.show()

sixth_problem_query0 = """
                        with cte_final_sixth as (
                        select
                        game_id,
                        event,
                        tournament_name,
                        move_no_pair as number_of_moves,
                        ROW_NUMBER()over(order by move_no_pair DESC)as high_rank_draw,
                        ROW_NUMBER()over(order by move_no_pair ASC)as low_rank_draw
                        from fifth_problem_query1
                        where winner like '%draw%' )
                        select
                        game_id,
                        event,
                        tournament_name,
                        number_of_moves
                        from cte_final_sixth
                        where high_rank_draw = 1 or low_rank_draw = 1
                        order by number_of_moves DESC"""

result_test6 = spark.sql(sixth_problem_query0)
result_test6.show()

# -----------------------7th Problem mitigation ----------------#

#chess_wc_history_movesTempTable.show()
#eco_codesTempTable.show()
#chess_wc_history_game_infoTempTable.show()

seventh_problem_query0 = """
                        with cte_Final_playerRanks as (
                        select
                        winner as player_name ,
                        winner_elo as elo,
                        ROW_NUMBER() over ( order by winner_elo DESC) as ranking_high,
                        ROW_NUMBER() over ( order by winner_elo ASC) as ranking_low
                        from chess_wc_history_game_infoTempTable1
                        where winner_elo is not null and loser_elo is not null )
                        select player_name, elo
                        from cte_Final_playerRanks
                        where ranking_high =1  or ranking_low = 1
                        order by elo DESC
"""

result_test7 = spark.sql(seventh_problem_query0)
result_test7.show()



+--------------------+---------------+
|              winner|tournament_name|
+--------------------+---------------+
|           Carlsen,M| WorldChamp2021|
|     Carlsen, Magnus| WorldChamp2016|
|     Carlsen, Magnus| WorldChamp2014|
|     Carlsen, Magnus| WorldChamp2013|
|           Gelfand,B| WorldChamp2012|
|             Anand,V| WorldChamp2010|
|             Anand,V| WorldChamp2008|
|             Anand,V| WorldChamp2007|
|           Kramnik,V| WorldChamp2006|
|              Leko,P| WorldChamp2004|
|           Kramnik,V| WorldChamp2000|
|      Kasparov, Gary| WorldChamp1990|
|      Kasparov, Gary| WorldChamp1987|
|      Kasparov, Gary| WorldChamp1986|
|      Kasparov, Gary| WorldChamp1985|
|     Karpov, Anatoly| WorldChamp1984|
|     Karpov, Anatoly| WorldChamp1981|
|     Karpov, Anatoly| WorldChamp1978|
|Fischer, Robert J...| WorldChamp1972|
|    Spassky, Boris V| WorldChamp1969|
+--------------------+---------------+
only showing top 20 rows

+--------------------+--------------+
